## Import Modules

In [1]:
import pdal
import sys
import os
import json
sys.path.append("../../../PythonScripts")
from glob import glob
import open3d as o3d
import os
import numpy as np
import pandas as pd
from multiprocessing import Pool
from pipeline_functions import downscale_las, get_scales, downscale_ply, getFeatures,getFeaturesParallel,Cylinder
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from time import time
# from scipy.spatial import cKDTree
from sklearn.neighbors import KDTree, BallTree
# from scipy.spatial import cKDTree
import pickle
from datetime import datetime, timezone
from itertools import repeat # Used for starmap function
import gc # Used to free up memory to prevent kernel restarting

sns.set()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Get number of processors to run loop

In [2]:
processors = os.cpu_count() - 4

## Get File to run

In [3]:
# ROOT = """/home/sspiegel/CapstoneData/WashingtonDC/OpenDataDC_LAS_Point_Cloud_2020_Block1/Block1/2117.las"""
ROOT = """/home/sspiegel/CapstoneData/Paris/training_10_classes/Lille1_1.ply"""
# ROOT = """/home/sspiegel/CapstoneData/PA/225019325.las"""

## Load in base point cloud

In [4]:
pipeline_json = [
    {
        "type": "readers.ply",
        "filename": ROOT
    }
        
]


pipeline = pdal.Pipeline(json.dumps(pipeline_json))

# Execute the pipeline
# This will process the data according to the stages defined in the pipeline
pipeline.execute()

xy = pipeline.arrays[0]

mta = pipeline.metadata



In [5]:
xyzT = np.array((xy["X"], xy["Y"], xy["Z"])).T

In [6]:
xyz = xyzT - np.min(xyzT, axis = 0)

## Get Classification

In [7]:
cls = xy["Classification"].astype(int)

In [8]:
scls = get_scales(S = 6) # Use base paprameters from function

In [9]:
scls

[(0.1, 0.02), (0.2, 0.04), (0.4, 0.08), (0.8, 0.16), (1.6, 0.32), (3.2, 0.64)]

## Instantiate point cloud

In [10]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)

In [11]:
pc_dict = []
for s in scls:
    
    pc = pcd.voxel_down_sample(s[1])
    # pc_array = np.asarray(pc.points)
    pc_dict.append({
      "r" : s[0],
     "grid_size" : s[1],
     "point_cloud" : pc
     # "point_cloud_array" : pc_carray
    })


## Get scaling factors

### Downscale point clouds: Methodology

* Downscale point clouds based on multiscaling factor
* The scaling factor chosen from [Thomas Hugues et. al](https://ieeexplore.ieee.org/document/8490990/)
* Parameters
  * initial radius ($r_0$): 0.1
  * Number of scales ($S$): 6
  * Base exponent for expanding radius ($\gamma$): 2
  * scaling factor of grid ($\rho$): 5
  * radius of sphere for scale $s$: $r_s = r_0 * \gamma^{s}$
  * downsampled voxel size: $\frac{r_s}{\rho}$

* Use multiprocessing to speed up process

### Sample point cloud (first radius)

In [12]:
test = pc_dict[4]

test_pc = test["point_cloud"]

cyl = Cylinder(R = test['r'])

tree = KDTree(test_pc.points)
# tree = cKDTree(CylPoints)

In [13]:
print(test)

{'r': 1.6, 'grid_size': 0.32, 'point_cloud': PointCloud with 647916 points.}


In [14]:
test_pc_arr = np.asarray(test_pc.points)

## Get Distance matrix

In [15]:
# cylPCDPoints = np.asarray(pcd.points)[:,:2]

In [16]:
distMat = tree.query_radius(pcd.points, r=test["r"])



# distMat = tree.query_ball_point(pcd.points, r=test["r"], workers = 1)


In [17]:
distMat.shape

(30033430,)

In [20]:
# args1 = zip(repeat(pcOut),repeat(test_pc_arr), distMat)
# with Pool(processes=processors) as pool1:
#     result1 = pool1.starmap(cyl.computePoints, args1)
# distMatFix = np.array(result1)

In [18]:
pcOut = np.asarray(pcd.points)


In [19]:
del pc_dict, xyzT, xyz, xy, pcd

gc.collect()

20

In [23]:
# cyl.computePointsParallel(pcOut[0],test_pc_arr,distMat[0])

In [23]:
DIV = 1000000
groups = distMat.shape[0] // DIV
distMatFix = []
# pcOutBig = []
for g in range(groups + 1):
    print("""Processing batch %d...""" % (g+1))
    practiceList = distMat[g*DIV:(g+1)*DIV]
    pcOutPut = pcOut[g*DIV:(g+1)*DIV,:]
    

    args1 = zip(pcOutPut,repeat(test_pc_arr), practiceList)
    with Pool(processes=processors) as pool1:
        result1 = pool1.starmap(cyl.computePointsParallel, args1)
    distMatFix.append(result1)
# distMatFix = np.asarray(distMatFix, dtype = "object")

Processing batch 1...
Processing batch 2...
Processing batch 3...
Processing batch 4...
Processing batch 5...
Processing batch 6...
Processing batch 7...
Processing batch 8...
Processing batch 9...
Processing batch 10...
Processing batch 11...
Processing batch 12...
Processing batch 13...
Processing batch 14...
Processing batch 15...
Processing batch 16...
Processing batch 17...
Processing batch 18...
Processing batch 19...
Processing batch 20...
Processing batch 21...
Processing batch 22...
Processing batch 23...
Processing batch 24...
Processing batch 25...
Processing batch 26...
Processing batch 27...
Processing batch 28...
Processing batch 29...
Processing batch 30...
Processing batch 31...


In [27]:
distMatFix = [np.asarray(d, dtype = "object") for d in distMatFix]

In [29]:
distMatFix = np.concatenate(distMatFix)

In [30]:
gc.collect()

681

In [ ]:
# distMat.dtype

In [ ]:
# distMatFix = np.asarray(result1, dtype = "object")

In [ ]:
# distMatFix[0]
# # 

In [ ]:
# distMat[0]

In [ ]:
# pcOut[0]

In [ ]:
# ids = np.random.randint(0, pcOut.shape[0], 1)

In [ ]:
# ids

In [ ]:
# pcCenter = o3d.geometry.PointCloud()
# pcDown = o3d.geometry.PointCloud()
# pcCyl = o3d.geometry.PointCloud()

# pcCenter.points = o3d.utility.Vector3dVector(pcOut[ids[0]].reshape(1,-1))
# pcDown.points = o3d.utility.Vector3dVector(test_pc_arr[distMat[ids[0]]])
# pcCyl.points = o3d.utility.Vector3dVector(test_pc_arr[distMatFix[ids[0]]])

# pcCenter.paint_uniform_color([1,0,0])
# pcDown.paint_uniform_color([0,1,0])
# pcCyl.paint_uniform_color([0,0,1])

# o3d.visualization.draw_geometries([pcCenter, pcDown, pcCyl])


In [33]:
del distMat,pcOutPut

In [34]:
gc.collect()

0

## Split and apply features algorithmKDTree

In [35]:
# DIV = 100000
# groups = di stMatFix.shape[0] // DIV

In [36]:
s = time()

bigList = []

# for g in range(groups + 1):
#     print("""Processing batch %d...""" % (g+1))
#     practiceList = distMatFix[g*DIV:(g+1)*DIV]

args = zip(repeat(test_pc_arr), distMatFix)
with Pool(processes=processors) as pool:
    bigList = pool.starmap(getFeaturesParallel, args)
    # result = np.array(bigList)
    # bigList.append(result)
    # print(practiceList.shape)
    # break

bigList = np.array(bigList)
e = time()

print("""\n\n\n\nDone!!!  Total time: %.4f seconds""" % (e - s))





Done!!!  Total time: 622.0208 seconds


In [37]:
bigList[:3,:]

array([[4.50831184e-01, 8.86793145e-02, 6.95266142e-01, 5.94017592e-01,
        3.47049866e-01, 5.89325390e-02, 4.02293248e-02, 9.22652984e-01,
        2.24770114e-01, 5.30000000e+01],
       [4.50831184e-01, 8.86793145e-02, 6.95266142e-01, 5.94017592e-01,
        3.47049866e-01, 5.89325390e-02, 4.02293248e-02, 9.22652984e-01,
        2.24770114e-01, 5.30000000e+01],
       [4.72926684e-01, 9.10300302e-02, 6.96747878e-01, 6.33674240e-01,
        3.10309035e-01, 5.60167217e-02, 3.93834274e-02, 9.43913276e-01,
        2.11830041e-01, 5.50000000e+01]])

## Save as a pickle file

In [38]:
del distMatFix, tree

gc.collect()

0

In [39]:
# pickleOut = f"""/home/sspiegel/CapstoneData/Paris/Toronto_3D/pickleFiles/{datetime.now(timezone.utc).strftime("%Y_%m_%dT%H_%M")}_L001_r_{str(test["r"]).split('.')[0]}_{str(test["r"]).split('.')[1]}_grid_{str(test["grid_size"]).split('.')[0]}_{str(test["grid_size"]).split('.')[1]}.pkl"""

In [40]:
getTime = datetime.now(timezone.utc).strftime("%Y_%m_%dT%H_%M")

featureOut = f"""/home/sspiegel/CapstoneData/Paris/training_10_classes/pickleFiles/cylinder/training/{getTime}_Lille1_1_cylinder_r_{str(test["r"]).split('.')[0]}_{str(test["r"]).split('.')[1]}_grid_{str(test["grid_size"]).split('.')[0]}_{str(test["grid_size"]).split('.')[1]}_features.npz"""
# pickleOutPC = f"""/home/sspiegel/CapstoneData/Paris/Toronto_3D/pickleFiles/{getTime}_L001_r_{str(test["r"]).split('.')[0]}_{str(test["r"]).split('.')[1]}_grid_{str(test["grid_size"]).split('.')[0]}_{str(test["grid_size"]).split('.')[1]}_pointcloud.pkl"""

# outDict = { "features" : bigList, "classification" : cls}
# # outDict = {"PointCloud" : pcOut}


# with open(pickleOut, 'wb') as f:
#     pickle.dump(outDict, f)

np.savez(featureOut,array1 = pcOut, array2 = bigList, array3 = cls)

## Save stats

In [41]:
sts = f"""/home/sspiegel/CapstoneData/Paris/training_10_classes/stats/{datetime.now(timezone.utc).strftime("%Y_%m_%d")}_cylinder_r_{str(test["r"]).split('.')[0]}_{str(test["r"]).split('.')[1]}_grid_{str(test["grid_size"]).split('.')[0]}_{str(test["grid_size"]).split('.')[1]}_features.json"""


In [42]:
jsStats = {
    "filename" : ROOT,
    "radius" : test["r"],
    "grid_size" : test["grid_size"],
    "total_time" : "%.4f seconds" % (e - s)
}

In [43]:
with open(sts, "w") as jsOut:
    jsOut.write(json.dumps(jsStats))
